In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from plato.planets.metrics import PlanetPopulationMetrics
from plato.planets.populations import PopulationModel
from plato.stars import filter_valid_targets
from plato.visualisation import FigureProcessor, get_palette, set_plot_defaults
from plato.utils import get_abspath

A new version of galpy (1.10.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [3]:
set_plot_defaults()

figure_directory = f"/figures/06_planet_populations/"
save = True

In [4]:
LOPS2 = pd.read_csv(get_abspath() + f"data/processed/LOPS2_targets.csv")
LOPN1 = pd.read_csv(get_abspath() + f"data/processed/LOPN1_targets.csv")

fields = pd.concat([LOPS2, LOPN1])
fields = filter_valid_targets(fields)
fields = fields[
    [
        "Radius",
        "Mass",
        "[Fe/H]",
        "[Fe/H]_apogee",
        "[Fe/H]_galah",
        "u1",
        "u2",
        "gaiaV",
        "n_cameras",
        "Population",
    ]
]

fields["cos_i"] = 0
fields["sigma_star"] = 10e-6
fields = fields.rename(
    columns={
        "Radius": "R_star",
        "Mass": "M_star",
        "gaiaV": "Magnitude_V",
    }
)

In [ ]:
metrics = PlanetPopulationMetrics()

## INDIVIDUAL POPULATION MOCKS

In [ ]:
for num_embryos in [10, 20, 50, 100]:
    population_model = PopulationModel(
        fields,
        num_embryos=num_embryos,
        additional_columns=["Population"],
    )

    mock = population_model.create_mock_observation(
        remove_zeros=False,
        draw_sample=False,
        add_planet_category=True,
        add_detection_efficiency=True,
        result_format="full",
        additional_columns=["Population"],
    )
    mock.to_csv(
        get_abspath() + f"data/interim/mock_populations/num_embryos={num_embryos}.csv",
        index=False,
    )

In [ ]:
mocks = {}
for num_embryos in [10, 20, 50, 100]:
    mocks[num_embryos] = pd.read_csv(
        get_abspath() + f"data/interim/mock_populations/num_embryos={num_embryos}.csv"
    )

In [ ]:
# overall occurence rates, for P < 500 days
occurence_rates = {}
for num_embryos, mock in mocks.items():
    mock = mock.copy()[mock["Porb"] < 500]
    occurence_rates[num_embryos] = len(mock) / len(fields)
pd.DataFrame.from_dict(
    occurence_rates, orient="index", columns=["Occurence Rate"]
).rename_axis("Number of Embryos")

,Occurence Rate
Number of Embryos,
10,2.642988
20,3.615099
50,4.995847
100,4.648871


In [ ]:
# median size of planets
average_sizes = {}
for num_embryos, mock in mocks.items():
    average_sizes[num_embryos] = mock["R_planet"].median()
pd.DataFrame.from_dict(
    average_sizes, orient="index", columns=["Median Radius"]
).rename_axis("Number of Embryos")

,Median Radius
Number of Embryos,
10,3.083269
20,2.954271
50,2.799274
100,2.060101
